In [7]:
# imports libs. Must be installed separately
import os
import time, datetime
import numpy as np
from matplotlib import pyplot as plt
from matplotlib import animation
from pygame import mixer

PI = np.pi
##


class Sound():
    def __init__(self, f_hz=200, length_seconds=5):
        self.f_sampling = 192000  # Hz
        self.length_seconds = length_seconds
        # f = 100 # BpM
        # f_hz = f / 60. # Hz
        self.f_hz = f_hz  # Hz

        self.omega = 2 * PI * self.f_hz
        self.dt = 1. / self.f_sampling

        self.generate_sound()

    def checker(self):
        # securities
        self.audio_array = np.array(self.audio_array)
        if self.time_array is None:  # t is the time array
            self.time_array = np.array(range(self.n_samples))
        if self.dt == 0:
            self.dt = self.time_array[1] - self.time_array[0]
        assert len(self.audio_array.shape) == 1, "cannot plot >1D arrays"

        self.n_samples = len(self.audio_array)


    def generate_sound(self):
        """ make a sound from scratch """

        # numérical time base :
        # makes an array from o to {length_seconds}, with step {1/f_echantillonnage}
        self.time_array = range(int(max(10, self.length_seconds * self.f_sampling)))
        self.time_array = np.array(self.time_array) / self.f_sampling  # 0 : 2/1000 : 2

        self.audio_array = np.real(np.exp(1j * self.omega * self.time_array))
        # x = (audio_array * 22000).astype(np.int32)
        # x = audio_array
        return (self.time_array, self.audio_array)


    def play_sound(self):

        # init
        self.checker()
        start_time = time.time()

        # make sound player
        mixer.init(frequency=self.f_sampling, channels=1)
        sound = mixer.Sound((np.real(self.audio_array)*2**32).astype(np.int32))

        # play sound
        sound.play()
        time.sleep(self.length_seconds)
        elapsed_time = time.time() - start_time
        print(f"Frequence: {self.f_hz}Hz \nLength of the sound : {datetime.timedelta(seconds=elapsed_time)}\n")


    def plot_sound(self):
        """
        play sound from a 1-D array
        :param audio_array: np.array, 1D (for now)
        :param t:
        :return:
        """
        # code security
        self.checker()

        # Fourier transform and frequency scale
        ft_array = np.fft.fft(self.audio_array, n=self.n_samples)
        ft_array = ft_array / max(ft_array)
        freq = np.fft.fftfreq(self.n_samples, d=self.dt) #* 60  # BPM
        phase = np.angle(ft_array[freq >= 0])

        # makes the graphs
        # time signal
        plt.figure()
        plt.subplot(221)
        n_points_to_plot = 1500
        plt.title(f"sampling_rate: {1/self.dt}Hz; "
                  f"n_points: {n_points_to_plot}")
        plt.grid()
        plt.plot(self.time_array[:n_points_to_plot], self.audio_array[:n_points_to_plot])
        plt.xlabel("time (s)")

        # fourier transforms (frequencies)
        plt.subplot(223)
        plt.title(f"abs( fft( sound ) )")
        plt.grid()
        plt.semilogx(freq[freq >= 0], np.abs(ft_array[freq >= 0]))
        plt.xlabel("freq (Hz)")

        # fourier transforms (frequencies)
        plt.subplot(222)
        plt.title(f"real( fft( sound ) )")
        plt.grid()
        plt.semilogx(freq[freq >= 0], np.real(ft_array[freq >= 0]))
        plt.xlabel("freq (Hz)")

        # phase of fourier transforms (frequencies)
        plt.subplot(224)
        plt.title(f"phase( fft( sound ) )")
        plt.grid()
        plt.semilogx(freq[freq >= 0], phase)
        plt.xlabel("freq (Hz)")


        plt.show()

        

In [8]:

    for f in [10, 50, 100, 200, 500, 1000]:
        x = Sound(f_hz=f, length_seconds=0.5)
        # x.plot_sound()
        x.play_sound()

Frequence: 10Hz 
Length of the sound : 0:00:00.500957

Frequence: 50Hz 
Length of the sound : 0:00:00.500798

Frequence: 100Hz 
Length of the sound : 0:00:00.500786

Frequence: 200Hz 
Length of the sound : 0:00:00.500865

Frequence: 500Hz 
Length of the sound : 0:00:00.500792

Frequence: 1000Hz 
Length of the sound : 0:00:00.501070

